## __02_Pleasant-lake-flopy-example_____Version__02__ <font color='red'> Sfr err </font>

## Trying to changing folders <font color='red'> __not_yet__ </font>

In [1]:
# pip install sfrmaker

In [2]:
import numpy as np                          # Environment Flopy
import matplotlib.pyplot as plt
import flopy
import flopy.mf6 as mf6
from flopy.discretization import StructuredGrid
from flopy.utils import Raster
import sfrmaker
import os
import shutil
# import Beta_dragon as hbd                                # ______________ new

In [3]:
org_d = os.path.join('..', 'data')                       # Path folder
Beta_ws = os.path.join('v01_Beta')                       # ______________ New folder
if os.path.exists(Beta_ws):
    shutil.rmtree(Beta_ws)
shutil.copytree(org_d,Beta_ws)

'v01_Beta'

In [4]:
model_name = 'B_pleasant'
# template_ws = os.path.join("v02_Beta")                    # New Folder !

In [5]:
sim = mf6.MFSimulation(sim_name=model_name, version="mf6", exe_name="mf6", sim_ws=Beta_ws)     # __.nam  
gwf = mf6.ModflowGwf(sim, modelname=model_name, save_flows=True)  # w = os.path.join('v01_Beta')   # if os.path.exists(w):   #    shutil.rmtree(w)

# __Basic__ __info__

In [6]:
nper, nlay, nrow, ncol = 1, 3, 60, 70                                       # t, z, y, x, 
delr, delc = 40, 40
xoffset, yoffset = 554400., 389200.0
epsg = 3070   # Wisconsin ... EPSG code 3070 
modelgrid = StructuredGrid(delr=np.ones(ncol) * delr,    delc=np.ones(nrow) * delc,      xoff=xoffset, yoff=yoffset, angrot=0)

In [7]:
br_surf = Raster.load('../data/br_surface.tif')                             #  
rs = br_surf.resample_to_grid(modelgrid, band=1, method='linear')           
np.savetxt('../data/botm_002.dat', rs)

In [8]:
top = [{'filename': '../data/top.dat'}]          # 04 surfaces, 03 layers
botm = [{'filename': '../data/botm_000.dat'},
        {'filename': '../data/botm_001.dat'},
        {'filename': '../data/botm_002.dat'}]
k = [{'filename': '../data/k_000.dat'},          # KH
     {'filename': '../data/k_001.dat'},
     {'filename': '../data/k_002.dat'}]
k33 = [{'filename': '../data/k33_000.dat'},      # KV
       {'filename': '../data/k33_001.dat'},
       {'filename': '../data/k33_002.dat'}]
strt = [top[0]] * nlay                           # Top for starting heads
recharge = {
    0: {'filename': '../data/rch_000.dat'}}      # RCH    
irch = [{'filename': '../data/irch.dat'}]        # iRCH   local
spec_head_perimeter = {
    0: {'filename': '../data/chd_000.dat'}}      # chd

# __Object__ __&__ __Model__ __Instancies__

In [9]:
gwf.modelgrid.set_coord_info(xoff=xoffset, yoff=yoffset, crs=epsg)                                                    # gwf.modelgrid
dis = mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=top, botm=botm, idomain=1)    # __.dis 
tdis = mf6.ModflowTdis(sim, time_units="days", nper=1, perioddata=[(1.0, 1, 1.0)])                                    # __.tdis  
ims = mf6.ModflowIms(sim, complexity="moderate", outer_dvclose=0.001)                                                 # Solver 

In [10]:
sim.write_simulation() # __________02

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing model B_pleasant...
    writing model name file...
    writing package dis...


In [26]:
oc = mf6.ModflowGwfoc(gwf, 
                      head_filerecord=f'{B_pleasant.name}.hds',
                      budget_filerecord=f'{B_pleasant.name}.cbc',
                      saverecord=[('head', 'all'), ("budget", "all")]) 

NameError: name 'B_pleasant' is not defined

In [ ]:
oc = mf6.ModflowGwfoc(gwf, 
                      head_filerecord=f'{gwf.name}.hds',                                                         # __.oc  Output  
                      budget_filerecord=f'{gwf.name}.cbc',
                      saverecord=[('head', 'all'), ("budget", "all")])        

In [ ]:
sim.write_simulation() # __________02

# __Aquifer__ __Properties__

In [ ]:
npf = mf6.ModflowGwfnpf(gwf, icelltype=1, k=k, k33=k33)                      # K

In [ ]:
ic = mf6.ModflowGwfic(gwf, strt=strt)                                        # initial conditions 

# <font color='green'> __Boundary__ __ Conditions__ </font>

In [ ]:
chd = mf6.ModflowGwfchd(gwf, stress_period_data=spec_head_perimeter)   # ___________ ok
rch = mf6.ModflowGwfrcha(gwf, recharge=recharge, irch=irch)

In [ ]:
lak = mf6.ModflowGwflak(gwf, boundnames=True, nlakes=1, connectiondata={'filename': '../data/lake_cn.dat'},
    packagedata=[[0, 290.85, 345, 'lake1']],
    perioddata={0: [[0, 'evaporation', 0.000715], [0, 'rainfall', 0.00209]]},
    surfdep=0.1)

In [ ]:
sim.write_simulation()

# <font color='red'> __Streamflow__ __err__ __!__</font>

# <font color='blue'> __Add__ __next__ __SFR__ __REPO__ </font>

In [ ]:
lines = sfrmaker.Lines.from_shapefile(shapefile='../data/edited_flowlines.shp', id_column='id', routing_column='toid',
    width1_column='width1', width2_column='width2', name_column='name', attr_length_units='meters')
sfrdata = lines.to_sfr(model=gwf, model_length_units='meters')
sfrdata.set_streambed_top_elevations_from_dem('../data/dem40m.tif', elevation_units='meters')
sfrdata.assign_layers()
sfr = sfrdata.create_mf6sfr(gwf)

In [ ]:
sim.write_simulation()
sim.run_simulation() 

## Postrocessing : Get_water_table()__Module ____` err_02 `

In [ ]:
from flopy.utils.postprocessing import get_water_table

In [ ]:
hds = gwf.output.head().get_data()

In [ ]:
wt = get_water_table(hds, nodata=-1e10)

In [ ]:
cbc = gwf.output.budget()
lak = cbc.get_data(text='lak', full3D=True)[0]
sfr = cbc.get_data(text='sfr', full3D=True)[0]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
pmv = flopy.plot.PlotMapView(gwf, ax=ax)
ctr = pmv.contour_array(
    wt, levels=np.arange(290, 315, 1), 
    linewidths=1, colors='b')
labels = pmv.ax.clabel(
    ctr, inline=True, 
    fontsize=8, inline_spacing=1)
vmin, vmax = -100, 100
im = pmv.plot_array(
    lak[0], cmap='coolwarm', 
    vmin=vmin, vmax=vmax)
im = pmv.plot_array(
    sfr.sum(axis=0), cmap='coolwarm', 
    vmin=vmin, vmax=vmax)
cb = fig.colorbar(
    im, shrink=0.5, label='Leakage, in m$^3$/day')
ax.set_ylabel("Northing, WTM meters")
ax.set_xlabel("Easting, WTM meters")
ax.set_aspect(1)
plt.tight_layout()
plt.savefig('results.pdf')